In [1]:
##定義讀檔及HRV特徵計算的方程式

import io
from numbers import Number
import re

import numpy as np
from scipy import interpolate


class EmptyFileError(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)


class FileNotSupportedError(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)


def open_rri(pathname_or_fileobj):
    if isinstance(pathname_or_fileobj, str):
        rri = _open_rri_from_path(pathname_or_fileobj)
    elif isinstance(pathname_or_fileobj, io.TextIOWrapper):
        rri = _open_rri_from_fileobj(pathname_or_fileobj)
    return _transform_rri(rri)


def _open_rri_from_path(pathname):
    if pathname.endswith('.txt'):
        with open(pathname, 'r') as fileobj:
            rri = _open_rri_from_fileobj(fileobj)
    elif pathname.endswith('.hrm'):
        with open(pathname, 'r') as fileobj:
            rri = _open_rri_from_fileobj(fileobj)
    else:
        raise FileNotSupportedError("File extension not supported")
    return rri


def _open_rri_from_fileobj(fileobj):
    file_content = fileobj.read()
    file_type = _identify_rri_file_type(file_content)
    if file_type == 'text':
        rri = _open_rri_from_text(file_content)
        if not rri:
            raise EmptyFileError('File without rri data')
    else:
        rri = _open_rri_from_hrm(file_content)
        if not rri:
            raise EmptyFileError('File without rri data')
    return rri


def _open_rri_from_text(file_content):##----------------------------------------
    rri = list(map(float,
                   re.findall(r'\S+' , file_content)))
    return rri


def _open_rri_from_hrm(file_content):
    rri_info_index = file_content.find('[HRData]')
    rri = None
    if rri_info_index >= 0:
        rri = list(map(float,
                       re.findall(r'\d+', file_content[rri_info_index:-1])))
    return rri


def _identify_rri_file_type(file_content):
    is_hrm_file = file_content.find('[HRData]')
    if is_hrm_file >= 0:
        file_type = 'hrm'
    else:
        rri_lines = file_content.split('\n')
        for line in rri_lines:
            current_line_number = re.findall(r'\d+', line)
            if current_line_number:
                if  current_line_number[0] == line.strip():
                    raise FileNotSupportedError('Text file not supported')
        file_type = 'text'
    return file_type


def validate_rri(func):
    def _validate(rri, *args, **kwargs):
        _validate_positive_numbers(rri)
        rri = _transform_rri(rri)
        return func(rri, *args, **kwargs)

    def _validate_positive_numbers(rri):
        if not all(map(lambda value: isinstance(value, Number) and value > 0,
                       rri)):
            raise ValueError('rri must be a list or numpy.ndarray of positive'
                             ' and non-zero numbers')

    return _validate


def _transform_rri(rri):
    rri = _transform_rri_to_miliseconds(rri)
    return np.array(rri)


def validate_frequency_domain_arguments(func):
    def _check_frequency_domain_arguments(rri, fs=4.0, method='welch',
                                          interp_method='cubic', **kwargs):
        _validate_available_methods(method)
        return func(rri, fs, method, interp_method, **kwargs)

    def _validate_available_methods(method):
        available_methods = ('welch', 'ar')
        if method not in available_methods:
            raise ValueError('Method not supported! Choose among: {}'.format(
                ', '.join(available_methods)))

    return _check_frequency_domain_arguments


def _create_time_info(rri):
    rri_time = np.cumsum(rri) / 1000.0  # make it seconds
    return rri_time - rri_time[0]   # force it to start at zero


def _transform_rri_to_miliseconds(rri):
    if np.median(rri) < 1:
        rri *= 1000
    return rri


def _interpolate_rri(rri, fs=4, interp_method='cubic'):
    if interp_method == 'cubic':
        return _interp_cubic_spline(rri, fs)
    elif interp_method == 'linear':
        return _interp_linear(rri, fs)


def _interp_cubic_spline(rri, fs):
    time_rri = _create_time_info(rri)
    time_rri_interp = _create_interp_time(rri, fs)
    tck = interpolate.splrep(time_rri, rri, s=0)
    rri_interp = interpolate.splev(time_rri_interp, tck, der=0)
    return time_rri_interp, rri_interp


def _interp_linear(rri, fs):
    time_rri = _create_time_info(rri)
    time_rri_interp = _create_interp_time(rri, fs)
    rri_interp = np.interp(time_rri_interp, time_rri, rri)
    return time_rri_interp, rri_interp


def _create_interp_time(rri, fs):
    time_rri = _create_time_info(rri)
    return np.arange(0, time_rri[-1], 1 / float(fs))

import numpy as np

from scipy.signal import welch
from spectrum import pburg



@validate_rri
def time_domain(rri):
    diff_rri = np.diff(rri)
    rmssd = np.sqrt(np.mean(diff_rri ** 2))
    sdnn = np.std(rri, ddof=1)  # make it calculates N-1
    nn50 = _nn50(rri)
    pnn50 = _pnn50(rri)
    mrri = np.mean(rri)
    mhr = np.mean(60 / (rri / 1000.0))

    return dict(zip(['rmssd', 'sdnn', 'nn50', 'pnn50', 'mrri', 'mhr'],
                    [rmssd, sdnn, nn50, pnn50, mrri, mhr]))


def _nn50(rri):
    return sum(abs(np.diff(rri)) > 50)


def _pnn50(rri):
    return _nn50(rri) / len(rri) * 100


@validate_frequency_domain_arguments
@validate_rri
def frequency_domain(rri, fs, method, interp_method=None, vlf_band=(0, 0.04),
                     lf_band=(0.04, 0.15), hf_band=(0.15, 0.4), **kwargs):
    if interp_method is not None:
        time_interp, rri = _interpolate_rri(rri, fs, interp_method)

    if method == 'welch':
        fxx, pxx = welch(x=rri, fs=fs, **kwargs)
    elif method == 'ar':
        fxx, pxx = _calc_pburg_psd(rri=rri,  fs=fs, **kwargs)

    return _auc(fxx, pxx, vlf_band, lf_band, hf_band)


def _auc(fxx, pxx, vlf_band, lf_band, hf_band):
    vlf_indexes = np.logical_and(fxx >= vlf_band[0], fxx < vlf_band[1])
    lf_indexes = np.logical_and(fxx >= lf_band[0], fxx < lf_band[1])
    hf_indexes = np.logical_and(fxx >= hf_band[0], fxx < hf_band[1])

    vlf = np.trapz(y=pxx[vlf_indexes], x=fxx[vlf_indexes])
    lf = np.trapz(y=pxx[lf_indexes], x=fxx[lf_indexes])
    hf = np.trapz(y=pxx[hf_indexes], x=fxx[hf_indexes])
    total_power = vlf + lf + hf
    lf_hf = lf / hf
    lfnu = (lf / (total_power - vlf)) * 100
    hfnu = (hf / (total_power - vlf)) * 100

    return dict(zip(['total_power', 'vlf', 'lf', 'hf', 'lf_hf', 'lfnu',
                    'hfnu'], [total_power, vlf, lf, hf, lf_hf, lfnu, hfnu]))


def _calc_pburg_psd(rri, fs, order=16, nfft=None):
    burg = pburg(data=rri, order=order, NFFT=nfft, sampling=fs)
    burg.scale_by_freq = False
    burg()
    return np.array(burg.frequencies()), burg.psd


@validate_rri
def non_linear(rri):
    sd1, sd2 = _poincare(rri)
    return dict(zip(['sd1', 'sd2'], [sd1, sd2]))


def _poincare(rri):
    diff_rri = np.diff(rri)
    sd1 = np.sqrt(np.std(diff_rri, ddof=1) ** 2 * 0.5)
    sd2 = np.sqrt(2 * np.std(rri, ddof=1) ** 2 - 0.5 * np.std(diff_rri,
                                                              ddof=1) ** 2)
    return sd1, sd2

In [4]:
##將輸入資料讀入，並進行簡單處理
#rri = open('C:/Users/wesleyhuan/Desktop/5.csv')
import csv
import numpy as np
import pandas as pd
list1 = []
f=open('C:/Users/wesleyhuan/Desktop/5.csv', 'r')#讀取的資料的位置及值

for i in csv.reader(f):
    try:
        for _ in i:
            list1.append(int(_))#將資料讀入成int
    except:
        pass
    
f.close()
#print(list1)#檢測存入資料

print(type(list1))#檢測存入資料是值
list2 = []#將資料進行超過最大紀錄值的處理
for i in range(0,len(list1)):
    if list1[i]==250:
        list2.append(list1[i]+list1[i+1])
    else:
        if list1[i-1]==250:
            pass
        else:
            list2.append(list1[i])

list3=[]#切割為2維(分別為每分鐘的PPI)
mincount=0
for i in range(0,len(list2)):
    if  sum(list2[0:i]) >= 12000*mincount:#建立好PPI的2維陣列
        list3.append([])
        mincount=mincount+1
    else:
        pass
        
    
    list3[int(sum(list2[0:i])/12000)].append(list2[i])#將每個數值放入完成最後list
        
mincount=0

list2=np.array(list2)

#print(list2)
#print(len(list1))
#print(len(list2))
#print(int(sum(list2)/12000))
#print(list3)



<class 'list'>


In [3]:
'''def rri_correct(rri):
    #定義好多、少的位置及校正後RRI的矩陣
    miss=[]
    extra=[]
    rri_corrected=[]
    
    for i in range(0,len(rri)): 
        if i < 12:
            med = sum(rri[0,24])/25
        elif i>len(rri)-12:
            med = sum(rri[end-24,end])/25
        else:
            med = sum(rri[i-12,i+12])/25
     
        if i==1:
            if rri[i] > 1.8*med:
                miss.append(i)
                irregularcount=irregularcount+1
            elif rri[i] < 1.8*med:
                extra.append(i)
                irregularcount=irregularcount+1
            else:
                
        else:
            pass
        
    
    
    
    return rri_corrected'''

'def rri_correct(rri):\n    #定義好多、少的位置及校正後RRI的矩陣\n    miss=[]\n    extra=[]\n    rri_corrected=[]\n    \n    for i in range(0,len(rri)): \n        if i < 12:\n            med = sum(rri[0,24])/25\n        elif i>len(rri)-12:\n            med = sum(rri[end-24,end])/25\n        else:\n            med = sum(rri[i-12,i+12])/25\n     \n        if i==1:\n            if rri[i] > 1.8*med:\n                miss.append(i)\n                irregularcount=irregularcount+1\n            elif rri[i] < 1.8*med:\n                extra.append(i)\n                irregularcount=irregularcount+1\n            else:\n                \n        else:\n            pass\n        \n    \n    \n    \n    return rri_corrected'